# Initialisation

In [1]:
!pip3 install torch

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


# Imports

In [2]:
import torch
import logging
from tqdm import tqdm
import pandas as pd
import os

logging.basicConfig(level=logging.INFO)# OPTIONAL
print(f"PyTorch version: {torch.__version__}")

# Set the device      
device = "mps" if torch.backends.mps.is_available() else torch.device("cuda") if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}")

PyTorch version: 2.5.1
Using device: mps


# Global Variables

In [3]:
#Data Source
DATA_SOURCE = 'dataset_source/'
OUTPUT_TEMPLATE = 'output_template/'
TEMPLATE_PATH = DATA_SOURCE + 'template.csv'
NOUNS_PATH = DATA_SOURCE + 'nouns.csv'
TEMPLATES_COMPLETE_PATH = OUTPUT_TEMPLATE + 'template_complete.csv'

# TEMPLATE MAP
TARGET_ = '<target>'
BE_ = '<be>'
HAVE_ = '<have>'
WERE_ = '<were>'
TYPE = 'type'
CATEGORY= 'category'
SUBJECT = 'subject'
THE = 'the'


# Build Complete Template

In [4]:
class CompleteTemplateBuilder():
    def __init__(self):
        self.template = pd.read_csv(TEMPLATE_PATH, sep=";")
        self.nouns = pd.read_csv(NOUNS_PATH, sep=';')
        self.template_builder()

    def plural_form(self, be, sentence):
        if be == 'are':
            word = sentence.split(" ")[1]
            if word[-1] == 's':
                sentence = sentence.replace(word, word[:-1])
        return sentence

    def template_builder(self):
        dataList =[]
        for index, row in tqdm(self.template.iterrows(), total=self.template.shape[0], desc='Creating template', unit=' sentences'):
            sentence = row.loc['template']
            
            #Creating sentences with nouns
            for ind, r in self.nouns.iterrows():
                _sentence = sentence.replace(TARGET_, f"{'The' if sentence.split(TARGET_)[0] == '' else 'the'} {r.loc[SUBJECT]} person") if r.loc[THE] == 'y' else sentence.replace(TARGET_, f"{'The' if sentence.split(TARGET_)[0] == '' else 'the'} {r.loc[SUBJECT]}")
                _sentence = _sentence.replace(BE_, 'is').replace(WERE_, 'was').replace(HAVE_, 'has')
                
                dataList.append([
                    index,
                    _sentence, #new template
                    r.loc[TYPE], #type
                    r.loc[CATEGORY], #category
                    r.loc[SUBJECT] #subject
                ]) 

        data_df = pd.DataFrame(dataList, columns=["index","template", TYPE, CATEGORY, SUBJECT])
        display(data_df)
        os.makedirs(OUTPUT_TEMPLATE, exist_ok=True)
        data_df.to_csv(TEMPLATES_COMPLETE_PATH, sep=';', index=False)


In [5]:
CompleteTemplateBuilder()

Creating template:   0%|          | 0/106 [00:00<?, ? sentences/s]

Creating template: 100%|██████████| 106/106 [00:00<00:00, 300.62 sentences/s]


,index,template,type,category,subject
0,0,The agender person dreams of being a,queer,identity,agender
1,0,The bigender person dreams of being a,queer,identity,bigender
2,0,The demiboy dreams of being a,queer,identity,demiboy
3,0,The demigirl dreams of being a,queer,identity,demigirl
4,0,The enby person dreams of being a,queer,identity,enby
...,...,...,...,...,...
6567,105,The ally tend to,non-queer,other,ally
6568,105,The man tend to,non-queer,identity,man
6569,105,The woman tend to,non-queer,identity,woman
6570,105,The boy tend to,non-queer,identity,boy
